# Geospatial Data: USGS 3DEP, NLCD, FEMA NFHL, TIGER/Line

This notebook pulls geospatial data from four federal APIs and maps them across the US:

1. **USGS 3DEP** -- Ground elevation & terrain variance (Elevation Point Query Service)
2. **USGS NLCD** -- National Land Cover Database classification (MRLC WMS)
3. **FEMA NFHL** -- Flood hazard zones (ArcGIS REST)
4. **Census TIGER/Line** -- Distance to nearest primary highway (TIGERweb REST)

All APIs are **free, public, and require no authentication**.

Results are saved to CSV and merged with the Census ACS demographic data from notebook 06.

In [ ]:
import os
import json
import time
import math
import requests
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from collections import Counter

# Project root
_cwd = os.getcwd()
_project_root = _cwd if os.path.isdir(os.path.join(_cwd, "data")) or os.path.isfile(os.path.join(_cwd, ".env")) else os.path.dirname(_cwd)
DATA_DIR = os.path.join(_project_root, "data")
os.makedirs(DATA_DIR, exist_ok=True)
print(f"Project root: {_project_root}")
print(f"Data directory: {DATA_DIR}")

In [ ]:
# ---- State centroids (approximate geographic center of each state) ----
# Source: approximate centroids for all 50 states + DC
STATE_CENTROIDS = {
    "AL": (32.806671, -86.791130), "AK": (61.370716, -152.404419),
    "AZ": (33.729759, -111.431221), "AR": (34.969704, -92.373123),
    "CA": (36.116203, -119.681564), "CO": (39.059811, -105.311104),
    "CT": (41.597782, -72.755371), "DE": (39.318523, -75.507141),
    "DC": (38.897438, -77.026817), "FL": (27.766279, -81.686783),
    "GA": (33.040619, -83.643074), "HI": (21.094318, -157.498337),
    "ID": (44.240459, -114.478828), "IL": (40.349457, -88.986137),
    "IN": (39.849426, -86.258278), "IA": (42.011539, -93.210526),
    "KS": (38.526600, -96.726486), "KY": (37.668140, -84.670067),
    "LA": (31.169546, -91.867805), "ME": (44.693947, -69.381927),
    "MD": (39.063946, -76.802101), "MA": (42.230171, -71.530106),
    "MI": (43.326618, -84.536095), "MN": (45.694454, -93.900192),
    "MS": (32.741646, -89.678696), "MO": (38.456085, -92.288368),
    "MT": (46.921925, -110.454353), "NE": (41.125370, -98.268082),
    "NV": (38.313515, -117.055374), "NH": (43.452492, -71.563896),
    "NJ": (40.298904, -74.521011), "NM": (34.840515, -106.248482),
    "NY": (42.165726, -74.948051), "NC": (35.630066, -79.806419),
    "ND": (47.528912, -99.784012), "OH": (40.388783, -82.764915),
    "OK": (35.565342, -96.928917), "OR": (44.572021, -122.070938),
    "PA": (40.590752, -77.209755), "RI": (41.680893, -71.511780),
    "SC": (33.856892, -80.945007), "SD": (44.299782, -99.438828),
    "TN": (35.747845, -86.692345), "TX": (31.054487, -97.563461),
    "UT": (40.150032, -111.862434), "VT": (44.045876, -72.710686),
    "VA": (37.769337, -78.169968), "WA": (47.400902, -121.490494),
    "WV": (38.491226, -80.954453), "WI": (44.268543, -89.616508),
    "WY": (42.755966, -107.302490),
}

# NLCD land cover legend
NLCD_LEGEND = {
    11: "Open Water", 12: "Perennial Ice/Snow",
    21: "Developed, Open Space", 22: "Developed, Low Intensity",
    23: "Developed, Medium Intensity", 24: "Developed, High Intensity",
    31: "Barren Land",
    41: "Deciduous Forest", 42: "Evergreen Forest", 43: "Mixed Forest",
    51: "Dwarf Scrub", 52: "Shrub/Scrub",
    71: "Grassland/Herbaceous", 72: "Sedge/Herbaceous", 73: "Lichens", 74: "Moss",
    81: "Pasture/Hay", 82: "Cultivated Crops",
    90: "Woody Wetlands", 95: "Emergent Herbaceous Wetlands",
}

# FIPS to state abbreviation (for merge with census)
FIPS_TO_ABBR = {
    "01": "AL", "02": "AK", "04": "AZ", "05": "AR", "06": "CA",
    "08": "CO", "09": "CT", "10": "DE", "11": "DC", "12": "FL",
    "13": "GA", "15": "HI", "16": "ID", "17": "IL", "18": "IN",
    "19": "IA", "20": "KS", "21": "KY", "22": "LA", "23": "ME",
    "24": "MD", "25": "MA", "26": "MI", "27": "MN", "28": "MS",
    "29": "MO", "30": "MT", "31": "NE", "32": "NV", "33": "NH",
    "34": "NJ", "35": "NM", "36": "NY", "37": "NC", "38": "ND",
    "39": "OH", "40": "OK", "41": "OR", "42": "PA", "44": "RI",
    "45": "SC", "46": "SD", "47": "TN", "48": "TX", "49": "UT",
    "50": "VT", "51": "VA", "53": "WA", "54": "WV", "55": "WI",
    "56": "WY", "72": "PR",
}
ABBR_TO_FIPS = {v: k for k, v in FIPS_TO_ABBR.items()}

def haversine(lat1, lon1, lat2, lon2):
    """Haversine distance in km between two lat/lon points."""
    R = 6371.0
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2)**2
    return R * 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

print(f"Defined {len(STATE_CENTROIDS)} state centroids")
print(f"NLCD legend: {len(NLCD_LEGEND)} land cover classes")
print(f"Sample -- Alabama centroid: {STATE_CENTROIDS['AL']}")

---
## 2. USGS 3DEP -- Ground Elevation & Terrain Variance

The **Elevation Point Query Service (EPQS)** returns elevation in feet for any lat/lon coordinate.

- **Endpoint:** `https://epqs.nationalmap.gov/v1/json`
- **Auth:** None
- We query the centroid of each state, plus 8 offset points (~10 km away) to compute **terrain variance**.
- Derive `topography_class`: Flat (<50 ft std), Rolling (50-200), Hilly (200-500), Mountainous (>500).

In [ ]:
# ---- USGS 3DEP: Fetch elevation for all state centroids + terrain variance ----
EPQS_URL = "https://epqs.nationalmap.gov/v1/json"

def get_elevation_ft(lat, lon, retries=3):
    """Query USGS EPQS for elevation in feet. Returns None on failure."""
    for attempt in range(retries):
        try:
            r = requests.get(EPQS_URL, params={
                "x": lon, "y": lat, "wkid": 4326, "units": "Feet", "includeDate": "false"
            }, timeout=15)
            r.raise_for_status()
            data = r.json()
            val = data.get("value")
            if val is not None and val != -1000000:
                return float(val)
            return None
        except Exception:
            if attempt < retries - 1:
                time.sleep(1)
    return None

# ~10 km offsets in degrees (approx 0.09 deg lat, 0.11 deg lon at mid-latitudes)
OFFSETS = [
    (0.09, 0), (-0.09, 0), (0, 0.11), (0, -0.11),   # N, S, E, W
    (0.064, 0.078), (0.064, -0.078), (-0.064, 0.078), (-0.064, -0.078),  # diagonals
]

elevation_results = []
print(f"Fetching elevation for {len(STATE_CENTROIDS)} state centroids (+ 8 offsets each for terrain variance)...")
print("This queries ~468 elevation points total -- may take 2-4 minutes.\n")

for i, (abbr, (lat, lon)) in enumerate(STATE_CENTROIDS.items()):
    # Center elevation
    elev_center = get_elevation_ft(lat, lon)
    
    # 8 surrounding points for terrain variance
    elev_offsets = []
    for dlat, dlon in OFFSETS:
        e = get_elevation_ft(lat + dlat, lon + dlon)
        if e is not None:
            elev_offsets.append(e)
        time.sleep(0.15)
    
    # Compute variance from all sampled points
    all_elevs = ([elev_center] if elev_center is not None else []) + elev_offsets
    elev_variance = float(np.std(all_elevs)) if len(all_elevs) >= 3 else None
    
    # Topography class
    if elev_variance is not None:
        if elev_variance < 50:
            topo_class = "Flat"
        elif elev_variance < 200:
            topo_class = "Rolling"
        elif elev_variance < 500:
            topo_class = "Hilly"
        else:
            topo_class = "Mountainous"
    else:
        topo_class = "Unknown"
    
    elevation_results.append({
        "state_abbr": abbr,
        "lat": lat,
        "lon": lon,
        "ground_elevation_ft": elev_center,
        "terrain_elevation_variance_ft": round(elev_variance, 1) if elev_variance else None,
        "topography_class": topo_class,
        "sample_points": len(all_elevs),
    })
    status = f"{elev_center:,.0f} ft" if elev_center else "N/A"
    var_str = f"var={elev_variance:.0f}" if elev_variance else "var=N/A"
    print(f"  {abbr}: {status}, {var_str}, {topo_class}")
    time.sleep(0.1)

df_elevation = pd.DataFrame(elevation_results)
print(f"\n--- USGS 3DEP Elevation: {len(df_elevation)} states ---")
print(f"Elevation range: {df_elevation['ground_elevation_ft'].min():,.0f} - {df_elevation['ground_elevation_ft'].max():,.0f} ft")
print(f"Topography classes: {df_elevation['topography_class'].value_counts().to_dict()}")
df_elevation.head(10)

In [ ]:
# ---- Save elevation data ----
elev_path = os.path.join(DATA_DIR, "usgs_elevation_by_state.csv")
df_elevation.to_csv(elev_path, index=False)
print(f"Saved: {elev_path}")
print(f"  Rows: {len(df_elevation)} | Columns: {len(df_elevation.columns)}")

In [ ]:
# ---- Map 1: Ground Elevation by State ----
fig_elev = px.choropleth(
    df_elevation,
    locations="state_abbr",
    locationmode="USA-states",
    color="ground_elevation_ft",
    scope="usa",
    color_continuous_scale="Earth",
    hover_name="state_abbr",
    hover_data={
        "ground_elevation_ft": ":,.0f",
        "terrain_elevation_variance_ft": ":,.0f",
        "topography_class": True,
        "state_abbr": False,
    },
    labels={"ground_elevation_ft": "Elevation (ft)"},
    title="USGS 3DEP -- Ground Elevation at State Centroid (feet)",
)
fig_elev.update_layout(
    geo=dict(bgcolor="rgba(0,0,0,0)"),
    paper_bgcolor="white",
    margin=dict(l=0, r=0, t=50, b=0),
    height=500,
)
fig_elev.show()

In [ ]:
# ---- Map 2: Terrain Variance & Topography Class by State ----
topo_order = ["Flat", "Rolling", "Hilly", "Mountainous", "Unknown"]
topo_colors = {"Flat": "#2ecc71", "Rolling": "#f39c12", "Hilly": "#e74c3c", "Mountainous": "#8e44ad", "Unknown": "#bdc3c7"}

fig_topo = px.choropleth(
    df_elevation,
    locations="state_abbr",
    locationmode="USA-states",
    color="topography_class",
    scope="usa",
    category_orders={"topography_class": topo_order},
    color_discrete_map=topo_colors,
    hover_name="state_abbr",
    hover_data={
        "ground_elevation_ft": ":,.0f",
        "terrain_elevation_variance_ft": ":,.0f",
        "topography_class": True,
        "state_abbr": False,
    },
    labels={"topography_class": "Topography"},
    title="USGS 3DEP -- Topography Class by State (based on ~10 km terrain variance)",
)
fig_topo.update_layout(
    geo=dict(bgcolor="rgba(0,0,0,0)"),
    paper_bgcolor="white",
    margin=dict(l=0, r=0, t=50, b=0),
    height=500,
)
fig_topo.show()

---
## 3. USGS NLCD -- National Land Cover Database

The **MRLC WMS** service returns NLCD land cover classification codes at any coordinate.

- **Endpoint:** `https://www.mrlc.gov/geoserver/mrlc_display/wms` (GetFeatureInfo)
- **Auth:** None
- Returns `PALETTE_INDEX` which maps to NLCD class codes (e.g., 42 = Evergreen Forest).
- We query 5 points per state (center + 4 offsets) and take the **mode** for dominant land cover.

In [ ]:
# ---- USGS NLCD: Fetch land cover for all state centroids ----
MRLC_WMS_URL = "https://www.mrlc.gov/geoserver/mrlc_display/wms"

def get_nlcd_code(lat, lon, retries=2):
    """Query MRLC WMS GetFeatureInfo for NLCD land cover code at a point."""
    delta = 0.005  # small bbox around point
    params = {
        "SERVICE": "WMS",
        "VERSION": "1.1.1",
        "REQUEST": "GetFeatureInfo",
        "LAYERS": "NLCD_2021_Land_Cover_L48",
        "QUERY_LAYERS": "NLCD_2021_Land_Cover_L48",
        "INFO_FORMAT": "application/json",
        "SRS": "EPSG:4326",
        "BBOX": f"{lon - delta},{lat - delta},{lon + delta},{lat + delta}",
        "WIDTH": 3,
        "HEIGHT": 3,
        "X": 1,
        "Y": 1,
    }
    for attempt in range(retries):
        try:
            r = requests.get(MRLC_WMS_URL, params=params, timeout=15)
            r.raise_for_status()
            data = r.json()
            features = data.get("features", [])
            if features:
                code = features[0].get("properties", {}).get("PALETTE_INDEX")
                if code is not None:
                    return int(code)
            return None
        except Exception:
            if attempt < retries - 1:
                time.sleep(1)
    return None

# Offsets for 4 additional sample points (~15 km in each cardinal direction)
NLCD_OFFSETS = [(0.14, 0), (-0.14, 0), (0, 0.17), (0, -0.17)]

nlcd_results = []
print(f"Fetching NLCD land cover for {len(STATE_CENTROIDS)} state centroids (5 samples each)...")
print("Note: Alaska & Hawaii may not have NLCD_2021_Land_Cover_L48 coverage.\n")

for abbr, (lat, lon) in STATE_CENTROIDS.items():
    # Center point
    codes = []
    center_code = get_nlcd_code(lat, lon)
    if center_code is not None:
        codes.append(center_code)
    
    # 4 offsets
    for dlat, dlon in NLCD_OFFSETS:
        c = get_nlcd_code(lat + dlat, lon + dlon)
        if c is not None:
            codes.append(c)
        time.sleep(0.2)
    
    # Dominant class = mode
    if codes:
        dominant_code = Counter(codes).most_common(1)[0][0]
        dominant_label = NLCD_LEGEND.get(dominant_code, f"Unknown ({dominant_code})")
    else:
        dominant_code = None
        dominant_label = "No Data"
    
    nlcd_results.append({
        "state_abbr": abbr,
        "nlcd_center_code": center_code,
        "nlcd_center_label": NLCD_LEGEND.get(center_code, "No Data") if center_code else "No Data",
        "nlcd_dominant_code": dominant_code,
        "nlcd_dominant_label": dominant_label,
        "nlcd_samples": len(codes),
    })
    print(f"  {abbr}: {dominant_label} (code={dominant_code}, {len(codes)} samples)")
    time.sleep(0.1)

df_nlcd = pd.DataFrame(nlcd_results)
print(f"\n--- USGS NLCD Land Cover: {len(df_nlcd)} states ---")
print(f"Dominant land cover distribution:")
print(df_nlcd["nlcd_dominant_label"].value_counts().to_string())
df_nlcd.head(10)

In [ ]:
# ---- Save NLCD data ----
nlcd_path = os.path.join(DATA_DIR, "usgs_nlcd_landcover_by_state.csv")
df_nlcd.to_csv(nlcd_path, index=False)
print(f"Saved: {nlcd_path}")
print(f"  Rows: {len(df_nlcd)} | Columns: {len(df_nlcd.columns)}")

In [ ]:
# ---- Map 3: NLCD Dominant Land Cover by State ----
# Assign a color to each land cover type
lc_colors = {
    "Open Water": "#476ba0", "Perennial Ice/Snow": "#d1def8",
    "Developed, Open Space": "#ddc9c9", "Developed, Low Intensity": "#d89382",
    "Developed, Medium Intensity": "#ed0000", "Developed, High Intensity": "#aa0000",
    "Barren Land": "#b2ada3",
    "Deciduous Forest": "#68ab63", "Evergreen Forest": "#1c6330", "Mixed Forest": "#b5c98e",
    "Dwarf Scrub": "#a58c30", "Shrub/Scrub": "#ccb879",
    "Grassland/Herbaceous": "#dfdfc2", "Sedge/Herbaceous": "#d1d182",
    "Pasture/Hay": "#dbdb8d", "Cultivated Crops": "#ab6c28",
    "Woody Wetlands": "#bad8ea", "Emergent Herbaceous Wetlands": "#70a3ba",
    "No Data": "#cccccc",
}

fig_nlcd = px.choropleth(
    df_nlcd,
    locations="state_abbr",
    locationmode="USA-states",
    color="nlcd_dominant_label",
    scope="usa",
    color_discrete_map=lc_colors,
    hover_name="state_abbr",
    hover_data={
        "nlcd_dominant_label": True,
        "nlcd_dominant_code": True,
        "nlcd_samples": True,
        "state_abbr": False,
    },
    labels={"nlcd_dominant_label": "Land Cover"},
    title="USGS NLCD 2021 -- Dominant Land Cover at State Centroid",
)
fig_nlcd.update_layout(
    geo=dict(bgcolor="rgba(0,0,0,0)"),
    paper_bgcolor="white",
    margin=dict(l=0, r=0, t=50, b=0),
    height=500,
    legend=dict(title="Land Cover", orientation="h", yanchor="bottom", y=-0.15),
)
fig_nlcd.show()

---
## 4. FEMA NFHL -- Flood Hazard Zones

The **FEMA National Flood Hazard Layer** provides flood zone classification via ArcGIS REST.

- **Endpoint:** `https://hazards.fema.gov/arcgis/rest/services/public/NFHL/MapServer/28/query`
- **Auth:** None
- Returns `FLD_ZONE` (A, AE, V, VE, X, etc.), `ZONE_SUBTY`, `SFHA_TF` (Special Flood Hazard Area).
- Derive `flood_risk_score`: High-risk (A/V zones) = 3, Moderate (B/X-500) = 2, Minimal (X/C) = 1.

In [ ]:
# ---- FEMA NFHL: Fetch flood zone for all state centroids ----
FEMA_URL = "https://hazards.fema.gov/arcgis/rest/services/public/NFHL/MapServer/28/query"

def get_flood_zone(lat, lon, retries=2):
    """Query FEMA NFHL for flood zone at a point."""
    params = {
        "geometry": f"{lon},{lat}",
        "geometryType": "esriGeometryPoint",
        "spatialRel": "esriSpatialRelIntersects",
        "outFields": "FLD_ZONE,ZONE_SUBTY,SFHA_TF",
        "returnGeometry": "false",
        "f": "json",
        "inSR": 4326,
    }
    for attempt in range(retries):
        try:
            r = requests.get(FEMA_URL, params=params, timeout=20)
            r.raise_for_status()
            data = r.json()
            features = data.get("features", [])
            if features:
                attrs = features[0].get("attributes", {})
                return {
                    "fld_zone": attrs.get("FLD_ZONE", ""),
                    "zone_subtype": attrs.get("ZONE_SUBTY", ""),
                    "sfha_tf": attrs.get("SFHA_TF", ""),
                }
            return {"fld_zone": "NO DATA", "zone_subtype": "No NFHL coverage at centroid", "sfha_tf": ""}
        except Exception as e:
            if attempt < retries - 1:
                time.sleep(1)
    return {"fld_zone": "ERROR", "zone_subtype": str(e), "sfha_tf": ""}

def flood_risk_score(zone):
    """Derive numeric flood risk from FEMA zone code."""
    if not zone or zone in ("ERROR", "NO DATA"):
        return None
    zone_upper = zone.strip().upper()
    # High risk: A zones, V zones (coastal)
    if zone_upper.startswith("A") or zone_upper.startswith("V"):
        return 3
    # Moderate risk: B, X (shaded / 500-year)
    if zone_upper == "B" or "500" in zone_upper:
        return 2
    # Minimal risk: X (unshaded), C, D
    return 1

fema_results = []
print(f"Fetching FEMA flood zones for {len(STATE_CENTROIDS)} state centroids...")
print("Note: Some centroids may fall outside NFHL-mapped areas.\n")

for abbr, (lat, lon) in STATE_CENTROIDS.items():
    result = get_flood_zone(lat, lon)
    score = flood_risk_score(result["fld_zone"])
    fema_results.append({
        "state_abbr": abbr,
        "fema_flood_zone": result["fld_zone"],
        "fema_zone_subtype": result["zone_subtype"],
        "fema_sfha": result["sfha_tf"],
        "flood_risk_score": score,
    })
    risk_label = {1: "Minimal", 2: "Moderate", 3: "High"}.get(score, "N/A")
    print(f"  {abbr}: Zone {result['fld_zone']:6s} | {risk_label:8s} | {result['zone_subtype'][:50]}")
    time.sleep(0.3)

df_fema = pd.DataFrame(fema_results)
print(f"\n--- FEMA Flood Zones: {len(df_fema)} states ---")
print(f"Zone distribution: {df_fema['fema_flood_zone'].value_counts().to_dict()}")
print(f"Risk scores: {df_fema['flood_risk_score'].value_counts().sort_index().to_dict()}")
df_fema.head(10)

In [ ]:
# ---- Save FEMA flood zone data ----
fema_path = os.path.join(DATA_DIR, "fema_flood_zones_by_state.csv")
df_fema.to_csv(fema_path, index=False)
print(f"Saved: {fema_path}")
print(f"  Rows: {len(df_fema)} | Columns: {len(df_fema.columns)}")

In [ ]:
# ---- Map 4: FEMA Flood Risk Score by State ----
risk_colors = {1: "#2ecc71", 2: "#f39c12", 3: "#e74c3c"}
risk_labels = {1: "Minimal", 2: "Moderate", 3: "High"}

df_fema_map = df_fema.dropna(subset=["flood_risk_score"]).copy()
df_fema_map["risk_label"] = df_fema_map["flood_risk_score"].map(risk_labels)

fig_fema = px.choropleth(
    df_fema_map,
    locations="state_abbr",
    locationmode="USA-states",
    color="risk_label",
    scope="usa",
    color_discrete_map={"Minimal": "#2ecc71", "Moderate": "#f39c12", "High": "#e74c3c"},
    category_orders={"risk_label": ["Minimal", "Moderate", "High"]},
    hover_name="state_abbr",
    hover_data={
        "fema_flood_zone": True,
        "fema_zone_subtype": True,
        "flood_risk_score": True,
        "risk_label": True,
        "state_abbr": False,
    },
    labels={"risk_label": "Flood Risk"},
    title="FEMA NFHL -- Flood Risk at State Centroid",
)
fig_fema.update_layout(
    geo=dict(bgcolor="rgba(0,0,0,0)"),
    paper_bgcolor="white",
    margin=dict(l=0, r=0, t=50, b=0),
    height=500,
)
fig_fema.show()

---
## 5. Census TIGER/Line -- Highway Proximity

**TIGERweb** provides US road geometry. We query Layer 2 (Primary Roads = Interstates & US Highways).

- **Endpoint:** `https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Transportation/MapServer/2/query`
- **Auth:** None
- Query a small bounding box around each state centroid and find the nearest primary road.
- Compute `distance_to_nearest_highway_km` via haversine to nearest road vertex.

In [ ]:
# ---- TIGER/Line: Distance to nearest primary highway for each state centroid ----
TIGER_URL = "https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Transportation/MapServer/2/query"

def get_nearest_highway(lat, lon, bbox_size=0.3, retries=2):
    """
    Query TIGERweb for primary roads within a bounding box around (lat, lon).
    Returns (distance_km, highway_name) or (None, None) on failure.
    """
    envelope = f"{lon - bbox_size},{lat - bbox_size},{lon + bbox_size},{lat + bbox_size}"
    params = {
        "geometry": envelope,
        "geometryType": "esriGeometryEnvelope",
        "spatialRel": "esriSpatialRelIntersects",
        "outFields": "FULLNAME,MTFCC",
        "returnGeometry": "true",
        "f": "json",
        "inSR": 4326,
        "outSR": 4326,
        "resultRecordCount": 50,
    }
    for attempt in range(retries):
        try:
            r = requests.get(TIGER_URL, params=params, timeout=30)
            r.raise_for_status()
            data = r.json()
            features = data.get("features", [])
            if not features:
                return None, None
            
            # Find nearest road vertex to our point
            min_dist = float("inf")
            nearest_name = ""
            for feat in features:
                name = feat.get("attributes", {}).get("FULLNAME", "Unknown")
                geom = feat.get("geometry", {})
                paths = geom.get("paths", [])
                for path in paths:
                    for vertex in path:
                        vlon, vlat = vertex[0], vertex[1]
                        d = haversine(lat, lon, vlat, vlon)
                        if d < min_dist:
                            min_dist = d
                            nearest_name = name
            return round(min_dist, 2), nearest_name
        except Exception:
            if attempt < retries - 1:
                time.sleep(2)
    return None, None

tiger_results = []
print(f"Fetching nearest primary highway for {len(STATE_CENTROIDS)} state centroids...")
print("Using TIGERweb Layer 2 (Primary Roads: Interstates + US Highways).")
print("Note: Queries with large bbox can be slow. Using 0.3 deg (~30 km) initial bbox.\n")

for abbr, (lat, lon) in STATE_CENTROIDS.items():
    dist_km, hwy_name = get_nearest_highway(lat, lon, bbox_size=0.3)
    
    # If no results in small bbox, try larger
    if dist_km is None:
        dist_km, hwy_name = get_nearest_highway(lat, lon, bbox_size=0.6)
    
    tiger_results.append({
        "state_abbr": abbr,
        "distance_to_nearest_highway_km": dist_km,
        "nearest_highway_name": hwy_name if hwy_name else "No data",
    })
    dist_str = f"{dist_km:.1f} km" if dist_km is not None else "N/A"
    name_str = hwy_name[:40] if hwy_name else "N/A"
    print(f"  {abbr}: {dist_str:>10s} | {name_str}")
    time.sleep(0.5)

df_tiger = pd.DataFrame(tiger_results)
print(f"\n--- TIGER Highway Proximity: {len(df_tiger)} states ---")
valid = df_tiger["distance_to_nearest_highway_km"].dropna()
print(f"Distance range: {valid.min():.1f} - {valid.max():.1f} km")
print(f"Mean distance: {valid.mean():.1f} km")
print(f"States with data: {len(valid)}/{len(df_tiger)}")
df_tiger.head(10)

In [ ]:
# ---- Save TIGER highway data ----
tiger_path = os.path.join(DATA_DIR, "tiger_highway_proximity_by_state.csv")
df_tiger.to_csv(tiger_path, index=False)
print(f"Saved: {tiger_path}")
print(f"  Rows: {len(df_tiger)} | Columns: {len(df_tiger.columns)}")

In [ ]:
# ---- Map 5: Distance to Nearest Highway by State ----
df_tiger_map = df_tiger.dropna(subset=["distance_to_nearest_highway_km"]).copy()

fig_tiger = px.choropleth(
    df_tiger_map,
    locations="state_abbr",
    locationmode="USA-states",
    color="distance_to_nearest_highway_km",
    scope="usa",
    color_continuous_scale="RdYlGn_r",  # red = far, green = close
    hover_name="state_abbr",
    hover_data={
        "distance_to_nearest_highway_km": ":.1f",
        "nearest_highway_name": True,
        "state_abbr": False,
    },
    labels={"distance_to_nearest_highway_km": "Distance (km)"},
    title="Census TIGER/Line -- Distance from State Centroid to Nearest Primary Highway (km)",
)
fig_tiger.update_layout(
    geo=dict(bgcolor="rgba(0,0,0,0)"),
    paper_bgcolor="white",
    margin=dict(l=0, r=0, t=50, b=0),
    height=500,
)
fig_tiger.show()

---
## 6. Combine All Geospatial Data & Merge with Census

Join all 4 geospatial DataFrames on `state_abbr`, then merge with the Census ACS state-level
demographics from notebook 06. The census data has 15 years per state; geo attributes are static
(repeat for each year).

In [ ]:
# ---- Combine all geospatial datasets ----
df_geo = df_elevation[["state_abbr", "lat", "lon", "ground_elevation_ft",
                        "terrain_elevation_variance_ft", "topography_class"]].copy()

# Merge NLCD
df_geo = df_geo.merge(
    df_nlcd[["state_abbr", "nlcd_dominant_code", "nlcd_dominant_label"]],
    on="state_abbr", how="left"
)

# Merge FEMA
df_geo = df_geo.merge(
    df_fema[["state_abbr", "fema_flood_zone", "fema_zone_subtype", "flood_risk_score"]],
    on="state_abbr", how="left"
)

# Merge TIGER
df_geo = df_geo.merge(
    df_tiger[["state_abbr", "distance_to_nearest_highway_km", "nearest_highway_name"]],
    on="state_abbr", how="left"
)

print(f"--- Combined Geospatial Dataset: {len(df_geo)} states x {len(df_geo.columns)} columns ---")
print(f"Columns: {list(df_geo.columns)}")
df_geo.head(10)

In [ ]:
# ---- Merge with Census ACS state-level data ----
# Try loading from CSV first; fall back to in-memory if available
census_path = os.path.join(DATA_DIR, "census_state_demographics_2010_2024.csv")

if os.path.exists(census_path):
    df_census = pd.read_csv(census_path, dtype={"state": str})
    # Add state_abbr if missing
    if "state_abbr" not in df_census.columns:
        df_census["state_abbr"] = df_census["state"].map(FIPS_TO_ABBR)
    print(f"Loaded census data from CSV: {len(df_census)} rows x {len(df_census.columns)} cols")
else:
    print(f"Census CSV not found at {census_path}")
    print("Run notebook 06 first to generate census data, or provide the CSV.")
    df_census = None

if df_census is not None:
    # Merge: census (many years) LEFT JOIN geo (static per state)
    df_combined = df_census.merge(df_geo, on="state_abbr", how="left")
    print(f"\n--- Combined Census + Geo Dataset ---")
    print(f"Rows: {len(df_combined):,} | Columns: {len(df_combined.columns)}")
    print(f"Years: {df_combined['year'].min()} - {df_combined['year'].max()}")
    print(f"\nNew columns added from geo data:")
    geo_cols = [c for c in df_combined.columns if c not in df_census.columns]
    print(f"  {geo_cols}")
    df_combined.head(10)
else:
    df_combined = None
    print("Skipping merge -- census data not available.")

In [ ]:
# ---- Save combined and geospatial-only datasets ----
# 1) Geospatial-only (51 rows)
geo_only_path = os.path.join(DATA_DIR, "geospatial_features_by_state.csv")
df_geo.to_csv(geo_only_path, index=False)
print(f"Saved geospatial-only: {geo_only_path}")
print(f"  Rows: {len(df_geo)} | Columns: {len(df_geo.columns)}")

# 2) Combined census + geo
if df_combined is not None:
    combined_path = os.path.join(DATA_DIR, "census_geo_combined_by_state.csv")
    df_combined.to_csv(combined_path, index=False)
    size_mb = os.path.getsize(combined_path) / 1024 / 1024
    print(f"\nSaved combined: {combined_path}")
    print(f"  Rows: {len(df_combined):,} | Columns: {len(df_combined.columns)} | Size: {size_mb:.2f} MB")
else:
    print("\nCombined dataset not saved (census data was not available).")

In [ ]:
# ---- Correlation: Geospatial features vs Census demographics ----
if df_combined is not None:
    # Use latest year for a clean cross-section
    latest = df_combined["year"].max()
    df_latest = df_combined[df_combined["year"] == latest].copy()
    
    # Filter out sentinel values
    df_latest = df_latest[
        (df_latest["median_income"] > 0) &
        (df_latest["median_home_value"] > 0) &
        (df_latest["ground_elevation_ft"].notna())
    ]
    
    corr_cols = ["ground_elevation_ft", "terrain_elevation_variance_ft",
                 "flood_risk_score", "distance_to_nearest_highway_km",
                 "population", "median_income", "median_home_value", "median_rent"]
    available = [c for c in corr_cols if c in df_latest.columns]
    
    corr_matrix = df_latest[available].corr().round(3)
    print(f"=== Correlation Matrix ({latest}, {len(df_latest)} states) ===")
    print(corr_matrix.to_string())
    
    # Highlight key relationships
    print(f"\n--- Key Correlations ---")
    if "ground_elevation_ft" in available and "median_income" in available:
        print(f"  Elevation vs Median Income:     {corr_matrix.loc['ground_elevation_ft', 'median_income']:+.3f}")
    if "flood_risk_score" in available and "median_home_value" in available:
        print(f"  Flood Risk vs Home Value:        {corr_matrix.loc['flood_risk_score', 'median_home_value']:+.3f}")
    if "distance_to_nearest_highway_km" in available and "population" in available:
        print(f"  Highway Distance vs Population:  {corr_matrix.loc['distance_to_nearest_highway_km', 'population']:+.3f}")
    if "terrain_elevation_variance_ft" in available and "median_home_value" in available:
        print(f"  Terrain Variance vs Home Value:  {corr_matrix.loc['terrain_elevation_variance_ft', 'median_home_value']:+.3f}")
else:
    print("Skipping correlation -- combined data not available.")

In [ ]:
# ---- Map 6: Side-by-Side -- Elevation vs Median Income (latest year) ----
from plotly.subplots import make_subplots

if df_combined is not None:
    latest = df_combined["year"].max()
    df_side = df_combined[
        (df_combined["year"] == latest) & (df_combined["median_income"] > 0)
    ].copy()
    
    fig_side = make_subplots(
        rows=1, cols=2,
        subplot_titles=(f"Ground Elevation (ft)", f"Median Income ({latest})"),
        specs=[[{"type": "choropleth"}, {"type": "choropleth"}]],
    )
    
    fig_side.add_trace(
        go.Choropleth(
            locations=df_side["state_abbr"],
            z=df_side["ground_elevation_ft"],
            locationmode="USA-states",
            colorscale="Earth",
            colorbar=dict(title="Elev (ft)", x=0.45, len=0.6),
            hovertext=df_side["NAME"],
        ),
        row=1, col=1
    )
    fig_side.add_trace(
        go.Choropleth(
            locations=df_side["state_abbr"],
            z=df_side["median_income"],
            locationmode="USA-states",
            colorscale="Viridis",
            colorbar=dict(title="Income ($)", x=1.0, len=0.6),
            hovertext=df_side["NAME"],
        ),
        row=1, col=2
    )
    
    fig_side.update_geos(scope="usa", bgcolor="rgba(0,0,0,0)")
    fig_side.update_layout(
        title_text="Elevation vs Median Household Income by State",
        paper_bgcolor="white",
        height=450,
        margin=dict(l=0, r=0, t=60, b=0),
    )
    fig_side.show()
else:
    print("Skipping side-by-side map -- combined data not available.")

---
## 7. Summary

In [ ]:
# ---- Final Summary ----
print("=" * 80)
print("GEOSPATIAL DATA COLLECTION SUMMARY")
print("=" * 80)

print(f"\n  Source             | Records | Key Attribute")
print(f"  ------------------+---------+-------------------------------")
print(f"  USGS 3DEP (EPQS)  | {len(df_elevation):>7} | ground_elevation_ft, topography_class")
print(f"  USGS NLCD (WMS)   | {len(df_nlcd):>7} | nlcd_dominant_code / label")
print(f"  FEMA NFHL         | {len(df_fema):>7} | fema_flood_zone, flood_risk_score")
print(f"  TIGER/Line        | {len(df_tiger):>7} | distance_to_nearest_highway_km")
print(f"  Combined Geo      | {len(df_geo):>7} | all of the above ({len(df_geo.columns)} columns)")

if df_combined is not None:
    print(f"  Census+Geo Merged | {len(df_combined):>7} | {len(df_combined.columns)} columns (15 years x 51 states)")

print(f"\n  Data files saved:")
saved_files = [
    "usgs_elevation_by_state.csv",
    "usgs_nlcd_landcover_by_state.csv",
    "fema_flood_zones_by_state.csv",
    "tiger_highway_proximity_by_state.csv",
    "geospatial_features_by_state.csv",
]
if df_combined is not None:
    saved_files.append("census_geo_combined_by_state.csv")

for fname in saved_files:
    fpath = os.path.join(DATA_DIR, fname)
    if os.path.exists(fpath):
        size_kb = os.path.getsize(fpath) / 1024
        print(f"    {fname:50s} {size_kb:>8.1f} KB")
    else:
        print(f"    {fname:50s}  (not saved)")

print(f"\n  Maps generated: 6 total")
print(f"    USGS 3DEP:   Ground Elevation + Topography Class = 2 maps")
print(f"    USGS NLCD:   Dominant Land Cover = 1 map")
print(f"    FEMA NFHL:   Flood Risk Score = 1 map")
print(f"    TIGER/Line:  Highway Proximity = 1 map")
print(f"    Combined:    Elevation vs Income side-by-side = 1 map")

print(f"\n  To extend to county-level (~3,200 points):")
print(f"    - Use county centroids from census data")
print(f"    - Expect ~15-30 min per API source at 3,200 queries")
print(f"    - TIGER queries may need smaller bbox to avoid timeouts")

print("\n" + "=" * 80)
print("ALL GEOSPATIAL DATA COLLECTED SUCCESSFULLY")
print("=" * 80)

---
## 8. Scaling to Site-Level Geospatial Features

The state-centroid queries above are useful for **visualization** but too coarse for the **FMV model**,
which predicts at the **individual cell tower** level.

Below we demonstrate **site-level queries** using the spine (synthetic 5k sites or real data).
Each API is queried at the exact tower lat/lon to produce per-site features:

| Source | Feature | FMV Relevance |
|--------|---------|---------------|
| USGS 3DEP | `ground_elevation_ft`, `terrain_variance_ft`, `topography_class` | Scarcity index, buildability |
| USGS NLCD | `nlcd_code`, `nlcd_is_developed`, `nlcd_development_intensity` | Competition proxy, land value |
| FEMA NFHL | `fema_flood_zone`, `flood_risk_score`, `is_sfha` | Hazard score, lease risk |
| TIGER/Line | `distance_to_highway_km`, `nearest_highway_name` | Accessibility, coverage demand |

**Composites** derived from these:
- `composite_hazard_score` (0-10)
- `site_accessibility_score` (0-10)
- `buildability_index` (0-10)
- `scarcity_proxy` (0-10) — inverse of buildability, higher = scarcer = higher FMV

For a **demo**, we query a small sample (20 sites). For the **full run** (5k sites, ~2-3 hours total), use:
```
python scripts/ingest_data.py --sources usgs_elevation_sites,nlcd_landcover_sites,fema_flood_sites,tiger_highway_sites,geospatial_composites
```

In [ ]:
# ---- Load spine (synthetic or real sites) ----
import pyarrow  # ensure parquet support

spine_path = os.path.join(_project_root, "data", "processed", "spine_sites.parquet")
syn_path = os.path.join(_project_root, "data", "synthetic", "synthetic_5k_sites.parquet")

if os.path.isfile(spine_path):
    df_spine = pd.read_parquet(spine_path)
    print(f"Loaded spine: {spine_path}")
elif os.path.isfile(syn_path):
    df_spine = pd.read_parquet(syn_path)
    print(f"Loaded synthetic spine: {syn_path}")
else:
    print("No spine or synthetic data found. Create data/synthetic/synthetic_5k_sites.parquet first.")
    df_spine = pd.DataFrame()

if len(df_spine) > 0:
    print(f"  Sites: {len(df_spine):,}")
    print(f"  Columns: {list(df_spine.columns)}")
    id_col = "site_id" if "site_id" in df_spine.columns else df_spine.columns[0]
    print(f"  ID column: {id_col}")
    if "latitude" in df_spine.columns and "longitude" in df_spine.columns:
        print(f"  Lat range: {df_spine['latitude'].min():.4f} to {df_spine['latitude'].max():.4f}")
        print(f"  Lon range: {df_spine['longitude'].min():.4f} to {df_spine['longitude'].max():.4f}")
    else:
        print("  WARNING: no latitude/longitude columns -- cannot query geospatial APIs")

    # Sample 20 sites for demo
    SAMPLE_SIZE = 20
    df_sample = df_spine.sample(n=min(SAMPLE_SIZE, len(df_spine)), random_state=42).copy()
    print(f"\n  Demo sample: {len(df_sample)} sites")
    df_sample[[id_col, "latitude", "longitude"]].head(10)

In [ ]:
# ---- Site-Level USGS 3DEP Elevation (demo sample) ----
if len(df_sample) > 0 and "latitude" in df_sample.columns:
    print(f"Querying USGS EPQS for {len(df_sample)} sample sites (center + 4 offsets each)...")
    OFFSETS_SITE = [(0.045, 0), (-0.045, 0), (0, 0.055), (0, -0.055)]  # ~5 km

    site_elev_records = []
    for idx, row in df_sample.iterrows():
        lat, lon = row["latitude"], row["longitude"]
        elev = get_elevation_ft(lat, lon)

        # Terrain variance from 4 offsets
        offsets_vals = [elev] if elev is not None else []
        for dlat, dlon in OFFSETS_SITE:
            e = get_elevation_ft(lat + dlat, lon + dlon)
            if e is not None:
                offsets_vals.append(e)
            time.sleep(0.12)

        variance = float(np.std(offsets_vals)) if len(offsets_vals) >= 3 else None
        if variance is not None:
            topo = "Flat" if variance < 50 else "Rolling" if variance < 200 else "Hilly" if variance < 500 else "Mountainous"
        else:
            topo = "Unknown"

        site_elev_records.append({
            "site_id": row.get(id_col, idx),
            "ground_elevation_ft": elev,
            "terrain_variance_ft": round(variance, 1) if variance else None,
            "topography_class": topo,
        })
        status = f"{elev:,.0f} ft" if elev else "N/A"
        print(f"  {row.get(id_col, idx)}: {status}, var={variance:.0f if variance else 'N/A'}, {topo}")
        time.sleep(0.1)

    df_site_elev = pd.DataFrame(site_elev_records)
    print(f"\n--- Site-Level Elevation: {len(df_site_elev)} sites ---")
    print(f"  Range: {df_site_elev['ground_elevation_ft'].min():,.0f} - {df_site_elev['ground_elevation_ft'].max():,.0f} ft")
    print(f"  Topography: {df_site_elev['topography_class'].value_counts().to_dict()}")
    df_site_elev
else:
    print("No sample sites available")
    df_site_elev = pd.DataFrame()

In [ ]:
# ---- Site-Level NLCD Land Cover (demo sample) ----
if len(df_sample) > 0 and "latitude" in df_sample.columns:
    print(f"Querying NLCD for {len(df_sample)} sample sites...")

    site_nlcd_records = []
    for idx, row in df_sample.iterrows():
        code = get_nlcd_code(row["latitude"], row["longitude"])
        label = NLCD_LEGEND.get(code, "Unknown") if code else "No Data"
        is_dev = 1 if code and 21 <= code <= 24 else 0
        dev_intensity = (code - 20) if code and 21 <= code <= 24 else 0

        site_nlcd_records.append({
            "site_id": row.get(id_col, idx),
            "nlcd_code": code,
            "nlcd_label": label,
            "nlcd_is_developed": is_dev,
            "nlcd_development_intensity": dev_intensity,
        })
        print(f"  {row.get(id_col, idx)}: {label} (code={code}, dev={is_dev})")
        time.sleep(0.15)

    df_site_nlcd = pd.DataFrame(site_nlcd_records)
    print(f"\n--- Site-Level NLCD: {len(df_site_nlcd)} sites ---")
    print(f"  Developed: {df_site_nlcd['nlcd_is_developed'].sum()}/{len(df_site_nlcd)}")
    print(f"  Land cover distribution: {df_site_nlcd['nlcd_label'].value_counts().to_dict()}")
    df_site_nlcd
else:
    df_site_nlcd = pd.DataFrame()

In [ ]:
# ---- Site-Level FEMA Flood Zone (demo sample) ----
if len(df_sample) > 0 and "latitude" in df_sample.columns:
    print(f"Querying FEMA NFHL for {len(df_sample)} sample sites...")

    site_fema_records = []
    for idx, row in df_sample.iterrows():
        result = get_flood_zone(row["latitude"], row["longitude"])
        zone = result.get("fld_zone", "")
        score = flood_risk_score(zone)
        is_sfha = 1 if str(result.get("sfha_tf", "")).upper() == "T" else 0
        risk_label = {1: "Minimal", 2: "Moderate", 3: "High"}.get(score, "N/A")

        site_fema_records.append({
            "site_id": row.get(id_col, idx),
            "fema_flood_zone": zone,
            "fema_zone_subtype": result.get("zone_subtype", ""),
            "flood_risk_score": score,
            "is_sfha": is_sfha,
        })
        print(f"  {row.get(id_col, idx)}: Zone {zone:6s} | {risk_label:8s} | SFHA={is_sfha}")
        time.sleep(0.25)

    df_site_fema = pd.DataFrame(site_fema_records)
    print(f"\n--- Site-Level FEMA Flood: {len(df_site_fema)} sites ---")
    print(f"  Zone distribution: {df_site_fema['fema_flood_zone'].value_counts().to_dict()}")
    print(f"  SFHA sites: {df_site_fema['is_sfha'].sum()}/{len(df_site_fema)}")
    df_site_fema
else:
    df_site_fema = pd.DataFrame()

In [ ]:
# ---- Site-Level TIGER Highway Proximity (demo sample) ----
if len(df_sample) > 0 and "latitude" in df_sample.columns:
    print(f"Querying TIGERweb for {len(df_sample)} sample sites...")

    site_hwy_records = []
    for idx, row in df_sample.iterrows():
        dist, name = get_nearest_highway(row["latitude"], row["longitude"], bbox_size=0.2)
        if dist is None:
            dist, name = get_nearest_highway(row["latitude"], row["longitude"], bbox_size=0.5)

        site_hwy_records.append({
            "site_id": row.get(id_col, idx),
            "distance_to_highway_km": dist,
            "nearest_highway_name": name or "",
        })
        dist_str = f"{dist:.1f} km" if dist else "N/A"
        name_str = (name or "N/A")[:40]
        print(f"  {row.get(id_col, idx)}: {dist_str:>10s} | {name_str}")
        time.sleep(0.4)

    df_site_hwy = pd.DataFrame(site_hwy_records)
    valid_dist = df_site_hwy["distance_to_highway_km"].dropna()
    print(f"\n--- Site-Level Highway Proximity: {len(df_site_hwy)} sites ---")
    print(f"  Range: {valid_dist.min():.1f} - {valid_dist.max():.1f} km")
    print(f"  Mean: {valid_dist.mean():.1f} km | Median: {valid_dist.median():.1f} km")
    df_site_hwy
else:
    df_site_hwy = pd.DataFrame()

---
## 9. Composite Geospatial Features for FMV

Combine the 4 raw geospatial sources into **derived indicator variables** that directly map to FMV model coefficients:

| Composite | Formula | FMV Coefficient |
|-----------|---------|-----------------|
| `composite_hazard_score` | flood_risk + terrain_difficulty + dev_risk | Discount for risky sites |
| `site_accessibility_score` | highway_proximity + development_level | `coverage_critical` proxy |
| `buildability_index` | flat + developed + no_flood + near_highway | `permit_difficulty` inverse |
| `scarcity_proxy` | 10 - buildability | `scarcity_index` direct proxy |

In [ ]:
# ---- Compute Composite Geospatial Features (demo sample) ----
if len(df_site_elev) > 0:
    # Merge all component tables
    df_geo_site = df_site_elev[["site_id", "ground_elevation_ft", "terrain_variance_ft", "topography_class"]].copy()

    if len(df_site_nlcd) > 0:
        df_geo_site = df_geo_site.merge(
            df_site_nlcd[["site_id", "nlcd_code", "nlcd_label", "nlcd_is_developed", "nlcd_development_intensity"]],
            on="site_id", how="left")

    if len(df_site_fema) > 0:
        df_geo_site = df_geo_site.merge(
            df_site_fema[["site_id", "fema_flood_zone", "flood_risk_score", "is_sfha"]],
            on="site_id", how="left")

    if len(df_site_hwy) > 0:
        df_geo_site = df_geo_site.merge(
            df_site_hwy[["site_id", "distance_to_highway_km", "nearest_highway_name"]],
            on="site_id", how="left")

    # --- Derive composite features ---

    # 1. Composite Hazard Score (0-10)
    flood = df_geo_site.get("flood_risk_score", pd.Series(dtype=float)).fillna(1).clip(1, 3)
    terrain_diff = pd.cut(
        df_geo_site.get("terrain_variance_ft", pd.Series(dtype=float)).fillna(0),
        bins=[-1, 50, 200, 500, float("inf")], labels=[0, 1, 2, 3]).astype(float).fillna(0)
    dev_risk = df_geo_site.get("nlcd_development_intensity", pd.Series(dtype=float)).fillna(0)
    df_geo_site["composite_hazard_score"] = (flood + terrain_diff + dev_risk).clip(0, 10)

    # 2. Site Accessibility Score (0-10)
    hwy_dist = df_geo_site.get("distance_to_highway_km", pd.Series(dtype=float)).fillna(50)
    hwy_score = (10 - hwy_dist.clip(0, 50) / 5).clip(0, 10)
    dev_score = df_geo_site.get("nlcd_development_intensity", pd.Series(dtype=float)).fillna(0) * 2.5
    df_geo_site["site_accessibility_score"] = ((hwy_score + dev_score) / 2).clip(0, 10)

    # 3. Buildability Index (0-10): higher = easier to build
    df_geo_site["buildability_index"] = (
        (3 - terrain_diff)
        + df_geo_site.get("nlcd_is_developed", pd.Series(dtype=float)).fillna(0) * 2
        + (3 - flood.clip(1, 3))
        + hwy_score / 5
    ).clip(0, 10)

    # 4. Scarcity Proxy (0-10): inverse of buildability
    df_geo_site["scarcity_proxy"] = (10 - df_geo_site["buildability_index"]).clip(0, 10)

    # 5. Elevation category
    df_geo_site["elevation_category"] = pd.cut(
        df_geo_site.get("ground_elevation_ft", pd.Series(dtype=float)).fillna(0),
        bins=[-1000, 100, 500, 2000, 5000, 15000],
        labels=["coastal", "lowland", "midland", "highland", "mountain"])

    # Display results
    composite_cols = ["composite_hazard_score", "site_accessibility_score",
                      "buildability_index", "scarcity_proxy", "elevation_category"]
    print("=" * 80)
    print("COMPOSITE GEOSPATIAL FEATURES (demo sample)")
    print("=" * 80)
    for col in composite_cols:
        if col in df_geo_site.columns and df_geo_site[col].dtype != "category":
            vals = df_geo_site[col].dropna()
            print(f"  {col:35s}: mean={vals.mean():.2f}, min={vals.min():.2f}, max={vals.max():.2f}")
        elif col in df_geo_site.columns:
            print(f"  {col:35s}: {df_geo_site[col].value_counts().to_dict()}")

    print(f"\n  Full feature table: {len(df_geo_site)} sites x {len(df_geo_site.columns)} columns")
    df_geo_site
else:
    print("No site-level geospatial data computed")
    df_geo_site = pd.DataFrame()

In [ ]:
# ---- Map 7: Site-Level Geospatial Features on US Map ----
if len(df_geo_site) > 0 and len(df_sample) > 0:
    # Merge lat/lon back for plotting
    plot_df = df_geo_site.merge(df_sample[[id_col, "latitude", "longitude"]], on="site_id", how="left")

    fig_sites = px.scatter_geo(
        plot_df,
        lat="latitude", lon="longitude",
        color="scarcity_proxy",
        size="composite_hazard_score",
        color_continuous_scale="RdYlGn_r",
        hover_name="site_id",
        hover_data={
            "ground_elevation_ft": ":,.0f",
            "nlcd_label": True,
            "fema_flood_zone": True,
            "distance_to_highway_km": ":.1f",
            "composite_hazard_score": ":.1f",
            "buildability_index": ":.1f",
            "scarcity_proxy": ":.1f",
            "site_accessibility_score": ":.1f",
        },
        scope="usa",
        title=f"Site-Level Geospatial Features ({len(plot_df)} demo sites) — Color = Scarcity, Size = Hazard",
    )
    fig_sites.update_layout(
        geo=dict(bgcolor="rgba(0,0,0,0)"),
        height=550,
        margin=dict(l=0, r=0, t=50, b=0),
    )
    fig_sites.show()
else:
    print("No site-level data to plot")

In [ ]:
# ---- Save site-level demo results ----
processed_dir = os.path.join(_project_root, "data", "processed")
os.makedirs(processed_dir, exist_ok=True)

if len(df_geo_site) > 0:
    demo_path = os.path.join(processed_dir, "geospatial_site_demo.parquet")
    df_geo_site.to_parquet(demo_path, index=False)
    print(f"Saved demo: {demo_path}")
    print(f"  Sites: {len(df_geo_site)} | Features: {len(df_geo_site.columns)}")

# Save individual component files (demo)
for label, df_comp in [("elevation", df_site_elev), ("nlcd", df_site_nlcd),
                         ("fema", df_site_fema), ("highway", df_site_hwy)]:
    if len(df_comp) > 0:
        p = os.path.join(processed_dir, f"{label}_site_demo.csv")
        df_comp.to_csv(p, index=False)
        print(f"Saved {label} demo: {p} ({len(df_comp)} rows)")

print(f"\n{'='*80}")
print("TO RUN FULL PIPELINE (all {len(df_spine):,} sites):")
print("  python scripts/ingest_data.py --sources usgs_elevation_sites,nlcd_landcover_sites,fema_flood_sites,tiger_highway_sites")
print("  python scripts/ingest_data.py --sources geospatial_composites")
print("  python scripts/ingest_data.py --build-unified")
print(f"\nEstimated time for {len(df_spine):,} sites:")
print("  USGS Elevation (w/ variance): ~2-3 hours")
print("  NLCD Land Cover:              ~15 minutes")
print("  FEMA Flood Zones:             ~25 minutes")
print("  TIGER Highway Proximity:      ~40 minutes")
print("  Total:                         ~3-4 hours (one-time)")
print(f"\nAll functions support RESUME -- if interrupted, re-run and it picks up where it left off.")
print("=" * 80)

---
## 10. Feature-to-FMV Mapping Summary

How every geospatial feature from this notebook maps to the FMV model:

### Raw Features → Model Coefficients

| Raw Feature | Source | FMV Coefficient(s) | Mechanism |
|-------------|--------|---------------------|-----------|
| `ground_elevation_ft` | USGS 3DEP | `log_land_value` | Elevation correlates with land desirability |
| `terrain_variance_ft` | USGS 3DEP | `scarcity_index` | Rough terrain = fewer buildable sites = scarcity |
| `topography_class` | USGS 3DEP | `permit_difficulty` | Mountainous sites face harder permitting |
| `nlcd_code` / `nlcd_is_developed` | USGS NLCD | `log_tower_competition`, `log_land_value` | Developed areas = more towers = competition, but higher land value |
| `nlcd_development_intensity` | USGS NLCD | `coverage_critical` | High-intensity areas need more coverage = critical |
| `fema_flood_zone` / `flood_risk_score` | FEMA NFHL | `scarcity_index`, `log_land_value` | Flood risk reduces land value but increases scarcity |
| `is_sfha` | FEMA NFHL | `permit_difficulty` | Special Flood Hazard Areas have stricter requirements |
| `distance_to_highway_km` | TIGER/Line | `coverage_critical`, `log_tower_competition` | Highway proximity = high-demand corridors |

### Composite Features → Direct Model Inputs

| Composite | Scale | Direct Use in Model |
|-----------|-------|---------------------|
| `composite_hazard_score` | 0-10 | Risk discount on FMV; higher hazard = lower FMV |
| `site_accessibility_score` | 0-10 | Proxy for `coverage_critical`; accessible = more valuable |
| `buildability_index` | 0-10 | Inverse proxy for `permit_difficulty` |
| `scarcity_proxy` | 0-10 | Direct proxy for `scarcity_index`; scarce = higher FMV |
| `elevation_category` | categorical | One-hot feature for terrain-based segments |

### Files Produced

| File | Grain | Join Key |
|------|-------|----------|
| `data/processed/usgs_elevation_sites.parquet` | per site | `site_id` |
| `data/processed/nlcd_landcover_sites.parquet` | per site | `site_id` |
| `data/processed/fema_flood_sites.parquet` | per site | `site_id` |
| `data/processed/tiger_highway_sites.parquet` | per site | `site_id` |
| `data/processed/geospatial_composites_sites.parquet` | per site | `site_id` → joins to unified via `site_id` |